# 🔌 GeckoTerminal API Client Initialization
GeckoTerminal provides comprehensive DEX data across multiple blockchain networks

In [ ]:
# 📦 Environment Setup and Path Configuration
# Add project root to Python path to enable imports from core modules
import warnings

warnings.filterwarnings("ignore")  # Suppress non-critical warnings for cleaner output

from geckoterminal_py import GeckoTerminalAsyncClient
import pandas as pd

# Create async client instance for efficient API calls
# This client handles rate limiting and provides structured access to pool data
gt = GeckoTerminalAsyncClient()

# ⚙️ Pool Filtering Configuration
These parameters define the criteria for identifying suitable trading pools

In [ ]:
# ============================================
# POOL AGE CONSTRAINTS (Risk Management)
# ============================================
MIN_POOL_AGE_DAYS = 7      # Minimum age to ensure pool stability and avoid rug pulls
MAX_POOL_AGE_DAYS = 720    # Maximum age to avoid obsolete or abandoned pools

# ============================================
# MARKET CAPITALIZATION RANGE (Mid-Cap Focus)
# ============================================
MIN_MARKET_CAP = 1_000_000     # $1M minimum for sufficient market depth
MAX_MARKET_CAP = 100_000_000   # $100M maximum to avoid mega-cap tokens (lower volatility)

# ============================================
# LIQUIDITY AND VOLUME REQUIREMENTS
# ============================================
MIN_VOLUME_24H = 100_000   # Minimum daily volume for active trading environment
MIN_LIQUIDITY = 10_000     # Minimum pool liquidity for efficient trade execution

# ============================================
# BLOCKCHAIN AND DEX SELECTION
# ============================================
NETWORK = "solana"          # Solana blockchain (fast, low-cost transactions)
DEX = "meteora"            # Meteora DEX (concentrated liquidity AMM)
QUOTE_ASSET = "SOL"        # SOL as quote currency for consistent pair analysis

print(f"🎯 Pool Selection Criteria:")
print(f"  • Pool Age: {MIN_POOL_AGE_DAYS}-{MAX_POOL_AGE_DAYS} days")
print(f"  • Market Cap: ${MIN_MARKET_CAP:,} - ${MAX_MARKET_CAP:,}")
print(f"  • Daily Volume: ${MIN_VOLUME_24H:,}+")
print(f"  • Pool Liquidity: ${MIN_LIQUIDITY:,}+")
print(f"  • Network: {NETWORK.title()}")
print(f"  • DEX: {DEX.title()}")
print(f"  • Quote Asset: {QUOTE_ASSET}")

In [ ]:
# 📥 Fetch Top Pools from Meteora DEX
# Retrieve the most active pools on Solana's Meteora exchange
pools = await gt.get_top_pools_by_network_dex(NETWORK, DEX)
print(f"📊 Retrieved {len(pools)} pools from {DEX.title()} on {NETWORK.title()}")

In [ ]:
# 🔧 Data Processing and Feature Engineering
# Convert raw pool data into analyzable format with additional metrics

# Convert string columns to numeric for mathematical operations
pools["market_cap_usd"] = pd.to_numeric(pools["market_cap_usd"])
pools["volume_usd_h24"] = pd.to_numeric(pools["volume_usd_h24"])
pools["reserve_in_usd"] = pd.to_numeric(pools["reserve_in_usd"])

# Parse datetime and remove timezone for consistent comparisons
pools["pool_created_at"] = pd.to_datetime(pools["pool_created_at"]).dt.tz_localize(None)

# Extract base and quote symbols from trading pair names
pools["base"] = pools["name"].apply(lambda x: x.split("/")[0].strip())    # Token being traded
pools["quote"] = pools["name"].apply(lambda x: x.split("/")[1].strip())   # Quote currency (SOL, USDC, etc.)

# Calculate volume-to-liquidity ratio (higher = more active trading relative to pool size)
pools["volume_liquidity_ratio"] = pools["volume_usd_h24"] / pools["reserve_in_usd"]

print(f"✅ Processed {len(pools)} pools with feature engineering")
print(f"📈 Volume/Liquidity ratio range: {pools['volume_liquidity_ratio'].min():.3f} - {pools['volume_liquidity_ratio'].max():.3f}")

In [ ]:
# 📊 Display Pool Data Overview
# Show processed pool data for analysis
print(f"📋 Pool Data Overview ({len(pools)} total pools):")
print(f"  • Columns: {', '.join(pools.columns)}")
print(f"  • Market Cap Range: ${pools['market_cap_usd'].min():,.0f} - ${pools['market_cap_usd'].max():,.0f}")
print(f"  • Volume Range: ${pools['volume_usd_h24'].min():,.0f} - ${pools['volume_usd_h24'].max():,.0f}")
print(f"  • Liquidity Range: ${pools['reserve_in_usd'].min():,.0f} - ${pools['reserve_in_usd'].max():,.0f}")

# Display sample of pool data
pools.head(10)

In [ ]:
# 🎯 Apply Pool Filtering Criteria
# Filter pools based on predefined criteria for trading suitability
from datetime import datetime, timedelta

# Calculate date boundaries for pool age filtering
min_pool_date_created = datetime.utcnow() - timedelta(days=MIN_POOL_AGE_DAYS)
max_pool_date_created = datetime.utcnow() - timedelta(days=MAX_POOL_AGE_DAYS)

# Apply comprehensive filtering criteria
pools_filtered = pools[
    (pools["market_cap_usd"] > MIN_MARKET_CAP) &                    # Market cap range
    (pools["market_cap_usd"] < MAX_MARKET_CAP) &
    (pools["volume_usd_h24"] > MIN_VOLUME_24H) &                    # Minimum volume
    (pools["reserve_in_usd"] > MIN_LIQUIDITY) &                     # Minimum liquidity
    (pools["pool_created_at"] < min_pool_date_created) &            # Pool age constraints
    (pools["pool_created_at"] > max_pool_date_created) &
    (pools["quote"] == QUOTE_ASSET)                                 # Quote asset filter
]

print(f"🔍 Pool Filtering Results:")
print(f"  • Initial pools: {len(pools)}")
print(f"  • After filtering: {len(pools_filtered)}")
print(f"  • Filtered out: {len(pools) - len(pools_filtered)} pools")
print(f"  • Filter efficiency: {len(pools_filtered)/len(pools)*100:.1f}%")

In [ ]:
# 🛠️ Price Analysis Functions
# Define functions for fetching candle data and calculating position metrics
from core.data_structures.candles import Candles

async def get_candles(pool, interval):
    """
    Fetch OHLCV candle data for a specific pool.
    
    Args:
        pool (pd.Series): Pool data containing base, quote, and address
        interval (str): Timeframe for candles (e.g., '1d', '4h', '1h')
    
    Returns:
        Candles: Processed candle data with trading pair information
    """
    trading_pair = f"{pool['base']}/{pool['quote']}"
    address = pool["address"]
    
    # Fetch OHLCV data from GeckoTerminal API
    ohlc = await gt.get_ohlcv(NETWORK, address, interval, currency="token")
    ohlc.index = pd.to_datetime(ohlc["timestamp"], unit="s")
    
    return Candles(candles_df=ohlc, connector_name="meteora", trading_pair=trading_pair, interval=interval)

def add_position_metrics(candle, lookback_periods):
    """
    Calculate position metrics for price range analysis.
    
    Args:
        candle (Candles): Candle data object
        lookback_periods (list): List of periods to analyze
    
    Returns:
        list: Position metrics for each lookback period
        
    Metrics calculated:
        - max_price: Highest price in period
        - min_price: Lowest price in period  
        - current_position: Where current price sits in range (0=low, 1=high)
        - range_price_pct: Price range as percentage of current price
    """
    df = candle.data
    results = []
    
    for period in lookback_periods:
        df_period = df.iloc[-period:]  # Get last N periods
        max_price = df_period['high'].max()
        min_price = df_period['low'].min()
        range_price = max_price - min_price
        current_price = df_period['close'].iloc[-1]
        
        # Calculate position in range (0 = at minimum, 1 = at maximum)
        current_position = (max_price - current_price) / range_price if range_price != 0 else 0
        
        # Price range as percentage of current price
        range_price_pct = (max_price - current_price) / current_price
        
        results.append({
            'period': period,
            'max_price': max_price,
            'min_price': min_price,
            'current_position': current_position,
            'range_price_pct': range_price_pct,
            'current_price': current_price
        })
    
    return results

print("✅ Price analysis functions defined")

In [ ]:
# 📈 Fetch Price Data and Calculate Metrics
# Download daily candle data for each filtered pool and calculate position metrics
pools_data = {}
interval = "1d"  # Daily timeframe for position analysis

print(f"📊 Analyzing {len(pools_filtered)} filtered pools...")
print(f"⏱️ Fetching {interval} candle data and calculating position metrics...")

for i, pool in pools_filtered.iterrows():
    try:
        # Fetch candle data for the pool
        candle = await get_candles(pool, interval)
        
        # Calculate position metrics using all available data
        position_metrics = add_position_metrics(candle, [len(candle.data)])[0]
        
        # Store pool data with analysis
        pools_data[pool["address"]] = {
            "candle": candle,
            "metrics": position_metrics,
            "pool": pool
        }
        
        print(f"  ✅ {pool['base']}/{pool['quote']}: {len(candle.data)} candles, position: {position_metrics['current_position']:.3f}")
        
    except Exception as e:
        print(f"  ❌ {pool['base']}/{pool['quote']}: Error - {str(e)}")
        continue

print(f"\n📊 Successfully analyzed {len(pools_data)} pools")

In [ ]:
# 📊 Sort Pools by Current Position
# Organize pools by how close they are to their recent highs (for reversal trading)
# Higher current_position = closer to recent high = potential reversal opportunity

pools_data = dict(sorted(pools_data.items(), key=lambda item: item[1]['metrics']['current_position'], reverse=True))

print(f"🔄 Sorted {len(pools_data)} pools by current position (highest to lowest)")
print(f"📈 Top pools near recent highs:")

for i, (pool_id, pool_data) in enumerate(list(pools_data.items())[:5]):
    metrics = pool_data["metrics"]
    pool_info = pool_data["pool"]
    pool_data["candle"].plot()
    print(f"  {i+1}. {pool_info['base']}/{pool_info['quote']}: Position {metrics['current_position']:.3f} (${metrics['current_price']:.6f})")